In [12]:
from django.test import TestCase
import json
import cv2
import numpy as np
import time
import os 
import torch
from SOM.UiComponent import UiComponent
from SOM import utils
from SOM import ip_draw 
import random
from SOM.sam import get_sam_gui_components_crops

In [23]:
def obtain_som(root_path, screenshots_names, image_folder_path, TIME_JSON_PATH, sam_checkpoint):
    COMPOS_JSON_PATH = 'compos_json/'
    MASK_JSON_PATH = 'mask_json/'

    for i,name in enumerate(screenshots_names):
        name = name.split('.')[0]
        recortes_path=root_path+'compos_npy/'+name+'.npy'
        path_to_save_mask_npy=root_path+'mask_elements_npy/'+name
        
        clips, uicompos, mask_json, compos_json, arrays_dict,dict_times = get_sam_gui_components_crops(param_img_root=image_folder_path, path_to_save_bordered_images=root_path, image_names=screenshots_names, img_index=i, checkpoint=sam_checkpoint)

        time0 = time.time()
        with open(COMPOS_JSON_PATH+name+'_'+sam_checkpoint+'.json','w') as f:
            f.write(compos_json)

        with open(MASK_JSON_PATH+name+'_'+sam_checkpoint+'.json','w') as f:
            f.write(mask_json)

        compos_aux = np.array(clips)
        np.save(recortes_path,compos_aux)

        # npy for each segmentation of SAM
        path=path_to_save_mask_npy
        for n in ['segmentation','crop_box']:
            path_element = path+'_'+n+'_'+sam_checkpoint+'.npy'
            aux = np.array(arrays_dict[n])
            np.save(path_element,aux)
        time1=time.time()
        
        dict_times['saving_results']=time1-time0

        times_json = json.dumps(dict_times)
        with open(TIME_JSON_PATH+name+'_'+sam_checkpoint+'.json','w') as f:
            f.write(times_json)

        print('There are {} components'.format(len(uicompos)))

In [24]:
screenshots_names = [
    "screenshot0012.JPEG",
    "screenshot0014.JPEG",
    "screenshot0015.JPEG",
    "screenshot0016.JPEG",
    "screenshot0017.JPEG",
    "screenshot0018.JPEG",
    "screenshot0019.JPEG",
    "screenshot0020.JPEG",
    "screenshot0022.JPEG",
    "screenshot0023.JPEG",
    "screenshot0024.JPEG",
    "screenshot0025.JPEG",
    "screenshot0026.JPEG"
]
root = "../../resources/gaze/gaze4/"
obtain_som(root, screenshots_names, root, root, "l")

FileNotFoundError: [Errno 2] No such file or directory: '/rim/apps/featureextraction/SOM/checkpoints/sam_vit_l_0b3195.pth'